In [266]:
from lxml import etree
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
import numpy as np
from PIL import ImageDraw
from random import randint

In [113]:
def get_bbox(path):
    path = 'Annotations/' + path
    tree = etree.parse(path)
    root = tree.getroot()
    xmin,ymin,xmax,ymax = [], [], [], []
    for j in root.findall('object'):
        for i in j.findall('bndbox'):
            xmin.append(i.find('xmin').text)
            ymin.append(i.find('ymin').text)
            ymax.append(i.find('ymax').text)
            xmax.append(i.find('xmax').text)
    return xmin, ymin, xmax, ymax

In [180]:
print xmin[1], ymin[1]
print zero_arr[xmin[1],ymin[1]]
print np.argpartition(zero_arr, 0).shape

1 235
1.0
(500, 335)


/Users/Ramana/projects/macvnev/lib/python2.7/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


In [181]:
def floodfill(matrix, x, y):
    #"hidden" stop clause - not reinvoking for "c" or "b", only for "a".
    if matrix[x][y] == 0:  
        matrix[x][y] = 2 
        #recursively invoke flood fill on all surrounding cells:
        if x > 0:
            floodfill(matrix,x-1,y)
        if x < len(matrix[y]) - 1:
            floodfill(matrix,x+1,y)
        if y > 0:
            floodfill(matrix,x,y-1)
        if y < len(matrix) - 1:
            floodfill(matrix,x,y+1)

In [119]:
def crop_image(xmin, ymin, xmax, ymax, j):
    image_path = 'JPEGImages/' +j
    original = Image.open(image_path)
    for i in range(0, len(xmin)):
        original.crop((int(xmin[i]), int(ymin[i]), int(xmax[i]), int(ymax[i]))).save("Cropped/"+ str(i) + "-" +str(j))

In [252]:
test_path = 'JPEGImages/2007_000042.jpg'
original = Image.open(test_path)
original.crop((0, 100, 450, 300)).show()

In [84]:
#Add all the XML files to the list
onlyfiles = []
path = 'Annotations/'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

In [121]:
k = 1
for j in onlyfiles:
    xmin, ymin, xmax, ymax = get_bbox(j)
    crop_image(xmin, ymin, xmax, ymax, j.split('.')[0]+".jpg")
    k += 1

ValueError: invalid literal for int() with base 10: '45.70000076293945'

In [127]:
#Testing Block
for k in range(14737, len(onlyfiles)-1):
    xmin, ymin, xmax, ymax = get_bbox(onlyfiles[k])
    crop_image(xmin, ymin, xmax, ymax, onlyfiles[k].split('.')[0]+".jpg")
    k += 1

In [191]:
bin_rand = np.random.randint(0,2,(40,40))


[[1 0]
 [0 1]]


In [286]:
for j in range(1453, len(onlyfiles)):
    xmin, xmax, ymin, ymax = get_bbox(onlyfiles[j])
    img_path = onlyfiles[j].split('.')[0]+ ".jpg"
    original = Image.open('JPEGImages/' + img_path)
    zero_arr = np.zeros(original.size)
    for i in range(len(xmin)):
        zero_arr[int(ymin[i]):int(ymax[i]), int(xmin[i]):int(xmax[i])] = 1
    xmin1,ymin1,xmax1,ymax1 = [],[],[],[]
    for i in range(zero_arr.shape[0]-200):
        for k in range(zero_arr.shape[1]-200):
            if np.count_nonzero(zero_arr[i:i+200, k:k+200])==0:
                xmin1.append(k)
                ymin1.append(i)
                xmax1.append(k+200)
                ymax1.append(i+200)
    n_back = 3
    if len(xmin1) < 3:
        n_back = len(xmin1)
    for ntemp in range(0,n_back):
        rand_int = randint(0, len(xmax1)-1)
        n_img = original.crop((xmin1[rand_int], ymin1[rand_int], xmax1[rand_int], ymax1[rand_int]))
        n_img.save('Background/' +str(ntemp)+ "-" +img_path)

KeyboardInterrupt: 

In [200]:
a = np.random.randint(0,3,(4,4))
print a

[[2 0 0 0]
 [0 0 2 1]
 [2 1 0 2]
 [0 0 2 1]]


In [250]:
a[1:3, 0:4]

array([[0, 0, 2, 1],
       [2, 1, 0, 2]])

In [276]:
#Add jpg files to list
back_files, obj_files = [], []
path1 = 'Background/'
back_files = [f for f in listdir(path1) if isfile(join(path1, f))]
path2 = 'Cropped/'
cropped_files = [f for f in listdir(path2) if isfile(join(path2, f))]

In [284]:
with open('train.txt', 'w+') as f2:
    for i in range(1, len(back_files)):
        f2.write(path1+back_files[i]+ " " +str(1))
        f2.write("\n")
        f2.write(path2+cropped_files[i] + " " +str(0))
        f2.write("\n")

In [285]:
print len(back_files)

1453


In [288]:
for jk in back_files[1:]:
    b_file = Image.open(path1+jk)
    b_file = b_file.resize((256, 256))
    b_file.save(path1+jk)

In [290]:
for kj in cropped_files[1:]:
    b_file = Image.open(path2+kj)
    b_file = b_file.resize((256, 256))
    b_file.save(path2+kj)